In [3]:
import os
import torch
import nest_asyncio
from IPython.display import Markdown, display
from PIL import Image
import base64
import ollama
from llama_index.core import Settings, VectorStoreIndex, PromptTemplate
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.schema import Document  # ✅ Import Document class

nest_asyncio.apply()

# ✅ Define the input directory
input_dir_path = "/teamspace/studios/this_studio/screenshots_output"

# ✅ Check if directory exists
if not os.path.exists(input_dir_path):
    raise ValueError(f"Directory {input_dir_path} does not exist!")

# ✅ Function to Convert Image to Base64 String
def encode_image_to_base64(image_path):
    """Encodes an image as a base64 string for text-based processing."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# ✅ Process Images & Store in Vector Index
docs = []
image_files = [f for f in os.listdir(input_dir_path) if f.endswith(".png")]

if len(image_files) == 0:
    raise ValueError(f"No PNG files found in {input_dir_path}.")

for image_file in image_files:
    image_path = os.path.join(input_dir_path, image_file)
    
    # ✅ Convert image to base64
    encoded_image = encode_image_to_base64(image_path)
    
    # ✅ Store in a Document for Retrieval
    doc = Document(
        text=f"Pokémon Red Image (Base64 Encoded): {encoded_image}",
        metadata={"image_path": image_path}
    )
    docs.append(doc)

# ✅ Load DeepSeek-V2 as the LLM
llm = Ollama(model="llama3.2-vision", request_timeout=120.0)
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5", trust_remote_code=True)

Settings.embed_model = embed_model
index = VectorStoreIndex.from_documents(docs, show_progress=True)

Settings.llm = llm
query_engine = index.as_query_engine()

# ✅ Modify Query Prompt to Use Image Data
qa_prompt_tmpl_str = (
    "You are analyzing Pokémon Red screenshots. "
    "Each image has been converted into a text-based format. "
    "Your task is to describe the image in **great detail**, including the Pokémon, locations, characters, and player actions. "
    "Use your best understanding of Pokémon Red to infer the scene from the provided image data."
    "\n\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Query: {query_str}\n"
    "Answer: "
)

qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

# ✅ Run the Query to Describe Images
query_text = "Describe three Pokémon Red images in great detail, using the provided image data."
response = query_engine.query(query_text)

print(response)
display(Markdown(str(response)))


Parsing nodes:   0%|          | 0/1000 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1952 [00:00<?, ?it/s]

HTTPStatusError: Client error '404 Not Found' for url 'http://localhost:11434/api/chat'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404

SyntaxError: invalid syntax (1133368903.py, line 1)